In [ ]:
import pandas as pd
import os
import re
from konlpy.tag import Okt

In [ ]:
path = '../data/crawling/'
file_list = os.listdir(path=path)
result_df = pd.DataFrame()
for file in file_list:
    if file[-3:]=='csv':
        temp_df = pd.read_csv(path+file)
        temp_df['캠퍼스'] = '인문' if file[:2]=='인문' else '자연'
        result_df = pd.concat([result_df, temp_df])
        print(file[:-4], len(result_df))

인문_1_리뷰 10000
인문_2_리뷰 18001
인문_3_리뷰 20103
인문캠(0~390) 41429
인문캠(464~600) 42782
자연2(1~250) 48174
자연2(251~500) 50760
자연2(501~750) 52112
자연2(751~) 52598
자연_1_리뷰 53815
자연캠(0~149) 58529
자연캠(150~513) 61608


In [ ]:
result_df['강의명길이'] = result_df['강의명'].apply(lambda x: len(x))
result_df.sort_values(by=['강의명길이', '강의명'], ascending=False).reset_index(drop=True, inplace=True)
result_df.head(2)

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,캠퍼스,강의명길이
0,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,인문,13
1,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,그저그래요. 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요. 비중도 ...,22년 2학기 수강자,4,0,인문,13


In [ ]:
result_df2 = result_df.copy()
result_df2.drop_duplicates(subset=['리뷰'], keep='first', inplace=True, ignore_index=False)
result_df2.head(2)

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,캠퍼스,강의명길이
0,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,인문,13
1,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,그저그래요. 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요. 비중도 ...,22년 2학기 수강자,4,0,인문,13


In [ ]:
result_df2.reset_index(drop=True, inplace=True)
result_df2.head(3)

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,캠퍼스,강의명길이
0,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,인문,13
1,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,그저그래요. 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요. 비중도 ...,22년 2학기 수강자,4,0,인문,13
2,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,"기창 강의평\n\n피드백 없고, 강의안도 없지만 그래도 할만한 꿀강...\n퀴즈 2...",22년 2학기 수강자,4,0,인문,13


In [ ]:
result_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44845 entries, 0 to 44844
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   강의명     44845 non-null  object
 1   학점      44845 non-null  int64 
 2   강좌유형    44845 non-null  object
 3   패논패여부   44845 non-null  object
 4   이수학년    44845 non-null  object
 5   전공여부    44845 non-null  object
 6   학과      44845 non-null  object
 7   교수명     44845 non-null  object
 8   리뷰      44845 non-null  object
 9   작성시점    44845 non-null  object
 10  평가별점수   44845 non-null  int64 
 11  공감수     44845 non-null  int64 
 12  캠퍼스     44845 non-null  object
 13  강의명길이   44845 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 4.8+ MB


In [ ]:
col_list = result_df2.columns.to_list()
for col in col_list:
    if result_df2[col].dtype == 'object':
        result_df2[col] = result_df2[col].astype('str').apply(lambda x: re.sub(r'[\n]', ' ', x))
        result_df2[col] = result_df2[col].astype('str').apply(lambda x: re.sub(r'[^a-zA-Z가-힣0-9/+-]', ' ', x))
        result_df2[col] = result_df2[col].astype('str').apply(lambda x: re.sub(r' +', ' ', x))
        result_df2[col] = result_df2[col].astype('object')

result_df2['리뷰길이'] = result_df2['리뷰'].apply(lambda x: len(x))

okt = Okt()
result_df2['리뷰_형태소_tuple'] = result_df2['리뷰'].apply(lambda x: okt.pos(x))

In [ ]:
result_df2.head(2)

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,캠퍼스,강의명길이,리뷰길이,리뷰_형태소_tuple
0,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,인문,13,28,"[(첫, Noun), (kcu, Alpha), (강의, Noun), (퀄리티, No..."
1,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,22년 2학기 수강자,4,0,인문,13,126,"[(그저, Adverb), (그래요, Adjective), (다른건, Adjecti..."


In [ ]:
result_df2['리뷰_형태소_list'] = result_df2['리뷰_형태소_tuple'].apply(lambda T: [t[0] for t in T])
result_df2['리뷰_형태소_only_N'] = result_df2['리뷰_형태소_tuple'].apply(lambda T: [t[0] for t in T if t[1]=='Noun'])
result_df2.head(3)

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,캠퍼스,강의명길이,리뷰길이,리뷰_형태소_tuple,리뷰_형태소_list,리뷰_형태소_only_N
0,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,인문,13,28,"[(첫, Noun), (kcu, Alpha), (강의, Noun), (퀄리티, No...","[첫, kcu, 강의, 퀄리티, 떨어지지만, 에, 이쁠, 받기, 쉬움]","[첫, 강의, 퀄리티, 쉬움]"
1,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,22년 2학기 수강자,4,0,인문,13,126,"[(그저, Adverb), (그래요, Adjective), (다른건, Adjecti...","[그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술, 형,...","[낫, 배드, 기말고사, 서술, 문제, 비중, 제일, 점수, 거, 만점, 말, 점, 못]"
2,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,22년 2학기 수강자,4,0,인문,13,226,"[(기, Modifier), (창, Noun), (강의, Noun), (평, Nou...","[기, 창, 강의, 평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한...","[창, 강의, 평, 피드백, 강의, 안도, 꿀강, 퀴즈, 번, 말, 시험, 번, 퀴..."


In [ ]:
result_df2.to_csv(path+'everytime_review_data_0227.csv', index=False, encoding='utf-8-sig')